In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/sanjidh090/dataset/unified_metadata.csv
/kaggle/input/datasets/sanjidh090/dataset/unified_metadata_cleaned_validated.csv
/kaggle/input/datasets/sanjidh090/dataset/unified_metadata_cleaned.csv
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0268_6977760_6997780.wav
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0051_1481919_1500020.wav
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0276_7150900_7171619.wav
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0262_6833520_6853460.wav
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0208_5428480_5450000.wav
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0216_5636639_5660100.wav
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0251_6516740_6527020.wav
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0175_4632800_4653219.wav
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0070_1963519_1983299.wav
/kaggle/input/datasets/sanjidh090/dataset/NFvZtFL37lc/0033_950870_971699.w

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("risalatlabib/bangla-audio-corrupted-30percent")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/datasets/risalatlabib/bangla-audio-corrupted-30percent


In [3]:
!pip install ctranslate2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 52.5 MB/s eta 0:00:00


In [4]:
pip install -U bitsandbytes>=0.46.1

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
from transformers import WhisperForConditionalGeneration, BitsAndBytesConfig
from peft import PeftModel, prepare_model_for_kbit_training

model_id = "hasans090/whisper-bengali-L40S-Final" # The base model
# This is the folder where you saved your adapter (e.g., from the previous step)
adapter_model_path = "/kaggle/input/datasets/fuadahm/tugstugi-adapter/whisper-medium-bn-final" 

# 1. Load Base Model in 8-bit
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False
)

model = WhisperForConditionalGeneration.from_pretrained(
    model_id, 
    quantization_config=bnb_config,
    device_map="auto"
)

# 2. Prepare for training (crucial for 8-bit)
model = prepare_model_for_kbit_training(model)

# 3. Load the EXISTING Adapter and make it trainable
# This connects your previous work instead of starting from scratch
model = PeftModel.from_pretrained(
    model, 
    adapter_model_path, 
    is_trainable=True
)

# Verify that parameters are trainable
model.print_trainable_parameters()

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/947 [00:00<?, ?it/s]

generation_config.json: 0.00B [00:00, ?B/s]

trainable params: 9,437,184 || all params: 773,295,104 || trainable%: 1.2204


In [6]:
import os
import random
import time
from pydub import AudioSegment
from pydub.scipy_effects import low_pass_filter
from tqdm import tqdm

# --- CONFIGURATION ---
# BASE_DIR = "/kaggle/working/Bangla_Natok_Dataset_Clean/data"
# CORRUPTION_RATE = 0.3   # 30% Target
# MINUTES_AGO = 30       # CONSIDER FILES MODIFIED IN LAST X MINS AS "ALREADY DONE"

# # Audio Settings
# HUMMING_CHANCE = 0.4
# CHUNK_LEN_MS = 1000 
# DEFAULT_CROSSFADE = 100

# --- EFFECT FUNCTIONS (Same as before) ---
def add_hall_reverb(sound):
    delays = [60, 150, 250, 350]
    gains =  [-6, -10, -14, -18]
    final_mix = sound - 2
    for delay, gain in zip(delays, gains):
        reflection = low_pass_filter(sound + gain, 1000) 
        final_mix = final_mix.overlay(reflection, position=delay)
    return final_mix

def apply_random_drifting(sound):
    final_output = AudioSegment.empty()
    total_len = len(sound)
    cursor = 0
    while cursor < total_len:
        end = min(cursor + CHUNK_LEN_MS, total_len)
        chunk = sound[cursor:end]
        if random.random() < HUMMING_CHANCE:
            processed_chunk = low_pass_filter(chunk, 600) - 8
        else:
            processed_chunk = low_pass_filter(chunk, 3000)
        
        if len(final_output) == 0:
            final_output = processed_chunk
        else:
            # Safe Crossfade to prevent crashes
            actual_crossfade = min(DEFAULT_CROSSFADE, len(processed_chunk) - 1)
            if actual_crossfade < 0: actual_crossfade = 0
            final_output = final_output.append(processed_chunk, crossfade=actual_crossfade)
        cursor += CHUNK_LEN_MS
    return final_output

# --- MAIN RECOVERY PROCESS ---
def process_dataset_safely():
    print(f"Scanning: {BASE_DIR}")
    print(f"Safety Check: Ignoring files modified in the last {MINUTES_AGO} minutes.")
    
    # Calculate cutoff time in seconds (Current time - X minutes)
    cutoff_time = time.time() - (MINUTES_AGO * 60)
    
    total_newly_corrupted = 0
    folders_scanned = 0

    for root, dirs, files in os.walk(BASE_DIR):
        wav_files = [f for f in files if f.lower().endswith('.wav')]
        if not wav_files:
            continue

        # --- SEPARATE CLEAN VS DIRTY FILES ---
        clean_candidates = []
        already_corrupted_count = 0
        
        for f in wav_files:
            full_path = os.path.join(root, f)
            mtime = os.path.getmtime(full_path)
            
            if mtime > cutoff_time:
                # File was modified RECENTLY (likely by your previous script)
                already_corrupted_count += 1
            else:
                # File is OLD (Clean)
                clean_candidates.append(f)

        # --- CALCULATE QUOTA ---
        total_files = len(wav_files)
        target_corruption_count = int(total_files * CORRUPTION_RATE)
        
        # How many MORE do we need to corrupt?
        needed = target_corruption_count - already_corrupted_count
        
        if needed > 0:
            print(f"\nFolder: {os.path.basename(root)}")
            print(f"   - Total Files: {total_files}")
            print(f"   - Already Corrupted: {already_corrupted_count}")
            print(f"   - Still Need to Corrupt: {needed}")
            
            # Select only from CLEAN candidates
            # If we need more than we have (rare), just take all clean ones
            count_to_process = min(needed, len(clean_candidates))
            files_to_process = random.sample(clean_candidates, count_to_process)
            
            for fname in tqdm(files_to_process, leave=False):
                full_path = os.path.join(root, fname)
                try:
                    audio = AudioSegment.from_file(full_path)
                    if len(audio) == 0: continue
                    
                    audio = add_hall_reverb(audio)
                    audio = apply_random_drifting(audio)
                    
                    # Overwrite
                    audio.export(full_path, format="wav")
                    total_newly_corrupted += 1
                    
                except Exception as e:
                    tqdm.write(f"⚠️ Error skipping {fname}: {e}")
                    continue
        else:
            # This folder is already ~30% corrupted (or more)
            pass
            
        folders_scanned += 1

    print("\n" + "="*30)
    print("       RESCUE JOB COMPLETED")
    print("="*30)
    print(f"Folders Scanned      : {folders_scanned}")
    print(f"Files Newly Corrupted: {total_newly_corrupted}")

# if __name__ == "__main__":
#     process_dataset_safely()

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [7]:
import os
import pandas as pd
from datasets import Dataset, Audio
from tqdm import tqdm

# --- CONFIG ---
INPUT_BASE_DIR = "/kaggle/input/datasets/risalatlabib/bangla-audio-corrupted-30percent/Bangla_Natok_Dataset_Clean/data"
METADATA_PATH = "/kaggle/input/datasets/risalatlabib/bangla-audio-corrupted-30percent/Bangla_Natok_Dataset_Clean/metadata.csv"
TARGET_HOURS = 15

# --- 1. LOAD & CLEAN METADATA ---
df = pd.read_csv(METADATA_PATH)
df.columns = df.columns.str.strip()

# Detect columns automatically
file_col = next((c for c in ['file_name', 'filename', 'wav_name', 'id'] if c in df.columns), None)
text_col = next((c for c in ['transcription', 'sentence', 'text'] if c in df.columns), None)

# --- 2. MAP PATHS USING DIRECTORY SCAN ---
# Since filenames alone don't reveal the folder, we map all existing files once
print("Mapping directory structure... this ensures all files are found.")
file_map = {}
for root, dirs, files in os.walk(INPUT_BASE_DIR):
    for file in files:
        if file.endswith(".wav"):
            file_map[file] = os.path.join(root, file)

# Apply the map to the metadata
df['audio'] = df[file_col].map(file_map)

# --- 3. FILTER AND SLICE ---
# Remove rows where audio wasn't found in the scan
initial_len = len(df)
df = df.dropna(subset=['audio']).copy()
print(f"Verified {len(df)} files exist (Filtered out {initial_len - len(df)} missing).")

# Select 25 hours
df = df[df['duration'] >= 1.2].sample(frac=1, random_state=42).reset_index(drop=True)
df['cum_sec'] = df['duration'].cumsum()
df_final = df[df['cum_sec'] <= (TARGET_HOURS * 3600)].copy()

print(f"Final training set: {len(df_final)} files ({df_final['duration'].sum()/3600:.2f} hours)")

# --- 4. CREATE HUGGINGFACE DATASET ---
# Rename text column to standard 'sentence' for Whisper
df_final = df_final.rename(columns={text_col: 'sentence'})
dataset = Dataset.from_pandas(df_final[['audio', 'sentence']])

# Cast the audio column to load on-the-fly during training
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# --- 5. PREPARE FOR TRAINING ---
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("hasans090/whisper-bengali-L40S-Final")

def prepare_dataset(batch):
    audio = batch["audio"]
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

# Map the processor (num_proc=2 is safer for Kaggle RAM)
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names, num_proc=1)

Mapping directory structure... this ensures all files are found.
Verified 2573 files exist (Filtered out 0 missing).
Final training set: 2568 files (11.66 hours)


processor_config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/2568 [00:00<?, ? examples/s]

In [8]:
dataset.shape

(2568, 2)

In [9]:
# INPUT_BASE_DIR = "/kaggle/input/datasets/sanjidh090/dataset"

# def get_full_path(fname):
#     # Logic: xO2c2H-ljc0001_... -> folder is xO2c2H-ljc
#     # We split by '0' to find the video ID prefix
#     video_id = fname.split('0')[0] 
#     return os.path.join(INPUT_BASE_DIR, video_id, fname)

# # Create the final dataframe with local paths
# df_final['audio'] = df_final[file_col].apply(get_full_path)
# df_final = df_final[['audio', text_col]]

# # Convert to Hugging Face Dataset
# dataset = Dataset.from_pandas(df_final)

# # IMPORTANT: This 'cast_column' tells the dataset to load the audio 
# # files on-demand during training at the correct 16kHz frequency.
# dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# print("Dataset successfully created and ready for training.")

In [10]:
dataset.shape

(2568, 2)

In [11]:
import torch

In [12]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Pad input_features (audio spectrograms)
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Pad labels (Bengali token IDs)
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding with -100 so it is ignored by the CrossEntropyLoss
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [13]:
model.print_trainable_parameters()

trainable params: 9,437,184 || all params: 773,295,104 || trainable%: 1.2204


In [14]:
!pip install -qU evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


In [15]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.1 MB/s eta 0:00:00


In [16]:

torch.cuda.empty_cache()

In [17]:
import evaluate
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# 1. Load the WER metric
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# 2. Define Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/whisper-medium-bn-checkpoints", # Explicit Kaggle path
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1, 
    learning_rate=1e-4,
    warmup_steps=100,
    max_steps=1000, 
    gradient_checkpointing=True,
    fp16=True,
    
    # Evaluation Strategy
    eval_strategy="steps",        # Change to "steps" to see WER
    eval_steps=300,               # Evaluate every 500 steps
    per_device_eval_batch_size=4,
    predict_with_generate=True,   # Required for WER calculation
    generation_max_length=225,
    
    # Checkpointing & Saving
    save_strategy="steps",
    save_steps=600,
    save_total_limit=2,           # Only keep the 2 best/latest checkpoints to save disk space
    load_best_model_at_end=True,  # Automatically load the best model based on WER
    metric_for_best_model="wer",
    greater_is_better=False,      # Lower WER is better
    
    logging_steps=25,
    dataloader_num_workers=4,
    report_to=["tensorboard"],
    remove_unused_columns=False,
    label_names=["labels"],
)

# 3. Initialize Trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset,        # Your 25h dataset
    eval_dataset=dataset.select(range(64)), # Use a small slice for fast eval
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor,
)

# 4. Start Training
trainer.train()

# 5. Save the final model/adapter
trainer.save_model("/kaggle/working/whisper-medium-bn-final")
processor.save_pretrained("/kaggle/working/whisper-medium-bn-final")

2026-02-20 07:18:49.934871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771571930.394151      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771571930.499417      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771571931.652230      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771571931.652263      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771571931.652266      23 computation_placer.cc:177] computation placer alr

Step,Training Loss,Validation Loss,Wer
300,0.038045,0.029964,2.330508
600,0.022705,0.018043,1.218220
900,0.014816,0.014339,1.006356


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensA

['/kaggle/working/whisper-medium-bn-final/processor_config.json']